# MLFlow

In [1]:
import pandas as pd
import numpy as np
import mlflow


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
import dagshub
dagshub.init(repo_owner='Pranay5519', repo_name='fraud_detection', mlflow=True)


Accessing as Pranay5519

Initialized MLflow to track repo "Pranay5519/fraud_detection"

Repository Pranay5519/fraud_detection initialized!

In [4]:
mlflow.set_tracking_uri("https://dagshub.com/Pranay5519/fraud_detection.mlflow")

In [5]:
df = pd.read_csv(r"D:\accredian\data\cleaned_fraud.csv")
df  = df[['step', 'type', 'isFraud', 'isMerchant',
       'orig_balance_diff', 'dest_balance_diff', 'log_amount']]

In [15]:

# ------------------------------
# 2. ENCODE CATEGORICAL COLUMN
# ------------------------------

df = pd.get_dummies(df, columns=['type'], drop_first=True)

# ------------------------------
# 3. DEFINE X AND y
# ------------------------------

X = df.drop(columns=['isFraud'])
y = df['isFraud']

# ------------------------------
# 4. TRAIN–TEST SPLIT
# ------------------------------

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# ------------------------------
# 5. SCALE FEATURES
# ------------------------------

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ------------------------------
# 6. UNDERSAMPLE (TRAINING DATA ONLY)
# ------------------------------
from imblearn.over_sampling import SMOTE

smote = SMOTE(
    sampling_strategy='auto',
    random_state=42,
    k_neighbors=5
)

X_train_bal, y_train_bal = smote.fit_resample(
    X_train_scaled,
    y_train
)

print("\nAfter SMOTE:")
print(y_train_bal.value_counts())

2026/01/16 15:32:40 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '0082918849274eefa75490de8d335162', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2026/01/16 15:32:40 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not have a `predict` or `transform` function, which is required in order to infer the signature
2026/01/16 15:32:42 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2026/01/16 15:32:49 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.


🏃 View run exultant-moose-502 at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/0/runs/0082918849274eefa75490de8d335162
🧪 View experiment at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/0

After SMOTE:
isFraud
0    5083526
1    5083526
Name: count, dtype: int64


In [16]:
# ------------------------------
# 7. TRAIN MODEL (PRECISION-FOCUSED)
# ------------------------------

class_weight = {0: 50, 1: 10}

lr = LogisticRegression(
    max_iter=1000,
    solver='lbfgs',
    class_weight=class_weight
)


In [17]:
# ------------------------------
# 7. START MLFLOW RUN
mlflow.sklearn.autolog()
mlflow.set_experiment("Fraud_Detection_LogisticRegression")
with mlflow.start_run(run_name="test run 3"):

    lr.fit(X_train_scaled, y_train)

    y_proba = lr.predict_proba(X_test_scaled)[:, 1]

    threshold = 0.9
    y_pred = (y_proba >= threshold).astype(int)

    # --------------------------
    # 8. METRICS
    # --------------------------

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)

    # --------------------------
    # 9. LOG PARAMETERS
    # --------------------------

    mlflow.log_param("model", "LogisticRegression")
    mlflow.log_param("class_weight", class_weight)
    mlflow.log_param("threshold", threshold)

    # --------------------------
    # 10. LOG METRICS
    # --------------------------

    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("roc_auc", roc_auc)

    # --------------------------
    # 11. LOG MODEL
    # --------------------------

    mlflow.sklearn.log_model(lr, "model")

    print("Run logged to MLflow")
    print(classification_report(y_test, y_pred, digits=4))

2026/01/16 15:34:05 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
d:\accredian\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\accredian\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
2026/01/16 15:34:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Run logged to MLflow
              precision    recall  f1-score   support

           0     0.9987    1.0000    0.9994   1270881
           1     0.0000    0.0000    0.0000      1643

    accuracy                         0.9987   1272524
   macro avg     0.4994    0.5000    0.4997   1272524
weighted avg     0.9974    0.9987    0.9981   1272524



d:\accredian\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\accredian\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\accredian\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


🏃 View run test run 3 at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/0/runs/7ecd27e423c7414aac30dcad2ab46574
🧪 View experiment at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/0


# Decision Tree

In [7]:
import pandas as pd
import mlflow
import mlflow.sklearn
import optuna

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

# ------------------------------
# 1. LOAD DATA
# ------------------------------

df = pd.read_csv(r"D:\accredian\data\cleaned_fraud.csv")
df.drop(columns=["Unnamed: 0"], inplace=True)

# ------------------------------
# 2. ENCODE CATEGORICAL COLUMN
# ------------------------------

df = pd.get_dummies(df, columns=['type'], drop_first=True)

# ------------------------------
# 3. DEFINE X AND y
# ------------------------------

X = df.drop(columns=['isFraud'])
y = df['isFraud']

# ------------------------------
# 4. TRAIN–TEST SPLIT
# ------------------------------

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# ------------------------------
# 5. SMOTE (TRAIN ONLY)
# ------------------------------

smote = SMOTE(random_state=42, k_neighbors=5)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)

# ------------------------------
# 6. MLFLOW + OPTUNA SETUP
# ------------------------------

mlflow.set_tracking_uri("https://dagshub.com/Pranay5519/fraud_detection.mlflow")
mlflow.set_experiment("Fraud_DecisionTree_Optuna Exp2")

mlflow.sklearn.autolog()

# ------------------------------
# 7. OPTUNA OBJECTIVE FUNCTION
# ------------------------------

def objective(trial):

    params = {
        "max_depth": trial.suggest_int("max_depth", 15, 150),
        "min_samples_split": trial.suggest_int("min_samples_split", 30, 100),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 40),
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
        "class_weight": trial.suggest_categorical(
            "class_weight",
            [{0: 5, 1: 1}, {0: 10, 1: 1}, {0: 7, 1: 1},{0: 12, 1: 1}]
        ),
        "random_state": 42
    }

    with mlflow.start_run(nested=True):

        model = DecisionTreeClassifier(**params)
        model.fit(X_train_bal, y_train_bal)

        y_test_pred = model.predict(X_test)

        # ---------- Classification Report ----------
        classification_rep = classification_report(
            y_test, y_test_pred, output_dict=True, zero_division=0
        )

        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Optuna optimization target → fraud recall
        return classification_rep["1"]["recall"]

# ------------------------------
# 8. RUN OPTUNA STUDY
# ------------------------------

with mlflow.start_run(run_name="DecisionTree_Run"):

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)

    mlflow.log_params({f"best_{k}": v for k, v in study.best_params.items()})

    best_params = study.best_params

    # --------------------------
    # 9. TRAIN FINAL MODEL
    # --------------------------

    final_model = DecisionTreeClassifier(**best_params, random_state=42)
    final_model.fit(X_train_bal, y_train_bal)

    y_test_pred = final_model.predict(X_test)

    classification_rep = classification_report(
        y_test, y_test_pred, output_dict=True, zero_division=0
    )

    for label, metrics in classification_rep.items():
        if isinstance(metrics, dict):
            for metric, value in metrics.items():
                mlflow.log_metric(f"final_{label}_{metric}", value)

    print("Best Params:", best_params)


2026/01/17 12:46:58 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '7630b52f00cc46ba97e506dbb555bfd0', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2026/01/17 12:46:58 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not have a `predict` or `transform` function, which is required in order to infer the signature
2026/01/17 12:46:59 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2026/01/17 12:47:06 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.


🏃 View run stately-robin-891 at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/7/runs/7630b52f00cc46ba97e506dbb555bfd0
🧪 View experiment at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/7


[I 2026-01-17 12:47:10,881] A new study created in memory with name: no-name-068e0489-8210-4ef5-867b-8608bde3e1e8
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 5, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 10, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 7, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices f

🏃 View run able-shoat-468 at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/7/runs/70950feba32845e78d6c118afa4e1418
🧪 View experiment at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/7


[I 2026-01-17 12:49:26,867] Trial 0 finished with value: 0.8618381010346926 and parameters: {'max_depth': 86, 'min_samples_split': 39, 'min_samples_leaf': 5, 'criterion': 'gini', 'class_weight': {0: 10, 1: 1}}. Best is trial 0 with value: 0.8618381010346926.
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 5, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 10, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0:

...

🏃 View run trusting-worm-375 at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/7/runs/b0f937525a944a79b709082303ee8b74
🧪 View experiment at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/7


[I 2026-01-17 13:51:06,372] Trial 28 finished with value: 0.8709677419354839 and parameters: {'max_depth': 105, 'min_samples_split': 30, 'min_samples_leaf': 21, 'criterion': 'entropy', 'class_weight': {0: 7, 1: 1}}. Best is trial 12 with value: 0.8849665246500305.
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 5, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 10, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contai

🏃 View run thoughtful-kit-307 at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/7/runs/5c05fddba9454a54822fd9fdadb759a6
🧪 View experiment at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/7


[I 2026-01-17 13:53:17,887] Trial 29 finished with value: 0.8667072428484479 and parameters: {'max_depth': 133, 'min_samples_split': 70, 'min_samples_leaf': 35, 'criterion': 'entropy', 'class_weight': {0: 10, 1: 1}}. Best is trial 12 with value: 0.8849665246500305.
2026/01/17 13:53:50 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\accredian\venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https:

Best Params: {'max_depth': 49, 'min_samples_split': 92, 'min_samples_leaf': 32, 'criterion': 'entropy', 'class_weight': {0: 5, 1: 1}}
🏃 View run DecisionTree_Run at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/7/runs/e5a8fbf61e04466d8f67d00d4dfb7a61
🧪 View experiment at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/7


# LogisticRegression

In [2]:
import dagshub
dagshub.init(repo_owner='Pranay5519', repo_name='fraud_detection', mlflow=True)


Accessing as Pranay5519

Initialized MLflow to track repo "Pranay5519/fraud_detection"

Repository Pranay5519/fraud_detection initialized!

In [ ]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
import optuna

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.metrics import (
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)
import tempfile
import os
# ------------------------------
# 1. LOAD DATA
# ------------------------------

df = pd.read_csv(r"D:\accredian\data\cleaned_fraud.csv")

# ------------------------------
# 2. ENCODE CATEGORICAL COLUMN
# ------------------------------

df = pd.get_dummies(df, columns=['type'], drop_first=True)

# ------------------------------
# 3. DEFINE X AND y
# ------------------------------

X = df.drop(columns=['isFraud'])
y = df['isFraud']

# ------------------------------
# 4. TRAIN–TEST SPLIT
# ------------------------------

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# ------------------------------
# 5. SCALE FEATURES
# ------------------------------

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ------------------------------
# 6. SMOTE (TRAIN ONLY)
# ------------------------------

smote = SMOTE(random_state=42, k_neighbors=5)

X_train_bal, y_train_bal = smote.fit_resample(
    X_train_scaled,
    y_train
)

# ------------------------------
# 7. MLFLOW + OPTUNA SETUP
# ------------------------------

mlflow.set_tracking_uri("https://dagshub.com/Pranay5519/fraud_detection.mlflow")
mlflow.set_experiment("Fraud_LogReg_Optuna")

mlflow.sklearn.autolog(disable=False)

# ------------------------------
# 8. OPTUNA OBJECTIVE FUNCTION
# ------------------------------

def objective(trial):

    params = {
        "C": trial.suggest_float("C", 0.001, 10.0, log=True),
        "solver": trial.suggest_categorical("solver", ["lbfgs", "liblinear"]),
        "class_weight": trial.suggest_categorical(
            "class_weight",
            [{0: 1, 1: 1}, {0: 2, 1: 1}, {0: 5, 1: 1}]
        ),
        "max_iter": 1000
    }

    with mlflow.start_run(nested=True):

        model = LogisticRegression(**params)
        model.fit(X_train_bal, y_train_bal)

        # ---------- Predictions ----------
        y_proba = model.predict_proba(X_test_scaled)[:, 1]
        y_pred = (y_proba >= 0.5).astype(int)

        # ---------- Metrics ----------
        roc_auc = roc_auc_score(y_test, y_proba)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)

        # ---------- Log metrics ----------
        mlflow.log_metric("test_roc_auc", roc_auc)
        mlflow.log_metric("test_precision", precision)
        mlflow.log_metric("test_recall", recall)
        mlflow.log_metric("test_f1_score", f1)

        # ---------- Confusion Matrix ----------
        cm = confusion_matrix(y_test, y_pred)

        # ---------- Save artifacts ----------
        with tempfile.TemporaryDirectory() as tmpdir:
            cm_path = os.path.join(tmpdir, "confusion_matrix.txt")
            cr_path = os.path.join(tmpdir, "classification_report.txt")

            with open(cm_path, "w") as f:
                f.write(str(cm))

            with open(cr_path, "w") as f:
                f.write(classification_report(y_test, y_pred, digits=4))

            mlflow.log_artifact(cm_path)
            mlflow.log_artifact(cr_path)

        return roc_auc
# ------------------------------
# 9. RUN OPTUNA STUDY
# ------------------------------

with mlflow.start_run(run_name="Log-Reg_run"):

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=3)

    mlflow.log_params(
        {f"best_{k}": v for k, v in study.best_params.items()}
    )
    best_params = study.best_params
    
    final_model = LogisticRegression(
                                **best_params,
                                max_iter=1000
                                    )

    final_model.fit(X_train_bal, y_train_bal)
    
    # Train
    y_train_proba = final_model.predict_proba(X_train_scaled)[:, 1]
    y_train_pred = (y_train_proba >= 0.5).astype(int)

    # Test
    y_test_proba = final_model.predict_proba(X_test_scaled)[:, 1]
    y_test_pred = (y_test_proba >= 0.5).astype(int)
    metrics = {
                # TRAIN
                "train_precision": precision_score(y_train, y_train_pred, zero_division=0),
                "train_recall": recall_score(y_train, y_train_pred, zero_division=0),
                "train_f1": f1_score(y_train, y_train_pred, zero_division=0),
                "train_roc_auc": roc_auc_score(y_train, y_train_proba),

                # TEST
                "test_precision": precision_score(y_test, y_test_pred, zero_division=0),
                "test_recall": recall_score(y_test, y_test_pred, zero_division=0),
                "test_f1": f1_score(y_test, y_test_pred, zero_division=0),
                "test_roc_auc": roc_auc_score(y_test, y_test_proba),
                }
    for k, v in metrics.items():
        mlflow.log_metric(k, v)
        
    mlflow.log_metric("best_roc_auc", study.best_value)

    print("Best ROC-AUC:", study.best_value)
    print("Best Params:", study.best_params)


d:\accredian\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2026-01-16 21:54:03,771] A new study created in memory with name: no-name-3b3d664b-278d-4872-a63d-9bae18476c5a
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 1, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 2, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribut

🏃 View run persistent-seal-209 at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/1/runs/ae614334d184440a9fc14014e126ca23
🧪 View experiment at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/1


[I 2026-01-16 21:55:09,340] Trial 0 finished with value: 0.9506688264865168 and parameters: {'C': 0.05892696682793512, 'solver': 'liblinear', 'class_weight': {0: 5, 1: 1}}. Best is trial 0 with value: 0.9506688264865168.
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 1, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 2, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 5, 1: 1} which is of type dict.
  warn

🏃 View run chill-steed-658 at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/1/runs/bf8aa477128d44f2b97801e97079e0ff
🧪 View experiment at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/1


[I 2026-01-16 21:56:17,137] Trial 1 finished with value: 0.9721546339248937 and parameters: {'C': 0.18614995516565452, 'solver': 'lbfgs', 'class_weight': {0: 5, 1: 1}}. Best is trial 1 with value: 0.9721546339248937.
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 1, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 2, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 5, 1: 1} which is of type dict.
  warnings

🏃 View run defiant-grouse-645 at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/1/runs/0cb4aa3aa314484d8737629ee672884e
🧪 View experiment at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/1


[I 2026-01-16 21:57:15,320] Trial 2 finished with value: 0.9372275226773534 and parameters: {'C': 0.012224662915859554, 'solver': 'lbfgs', 'class_weight': {0: 5, 1: 1}}. Best is trial 1 with value: 0.9721546339248937.


Best ROC-AUC: 0.9721546339248937
Best Params: {'C': 0.18614995516565452, 'solver': 'lbfgs', 'class_weight': {0: 5, 1: 1}}
🏃 View run Log-Reg_run at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/1/runs/ba719e181484446fa53b71906dd889fc
🧪 View experiment at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/1


# Random Forest Undersampling

In [1]:
import dagshub
dagshub.init(repo_owner='Pranay5519', repo_name='fraud_detection', mlflow=True)


Accessing as Pranay5519

Initialized MLflow to track repo "Pranay5519/fraud_detection"

Repository Pranay5519/fraud_detection initialized!

In [ ]:
import pandas as pd
import mlflow
import mlflow.sklearn
import optuna
from mlflow.models import infer_signature
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler

# ------------------------------
# 1. LOAD DATA
# ------------------------------

df = pd.read_csv(r"D:\accredian\data\cleaned_fraud.csv")
df.drop(columns=["Unnamed: 0"], inplace=True)
# ------------------------------
# 2. ENCODE CATEGORICAL COLUMN
# ------------------------------

df = pd.get_dummies(df, columns=['type'], drop_first=True)

# ------------------------------
# 3. DEFINE X AND y
# ------------------------------

X = df.drop(columns=['isFraud'])
y = df['isFraud']

# ------------------------------
# 4. TRAIN–TEST SPLIT
# ------------------------------

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# ------------------------------
# 5. SCALE FEATURES
# (Not required for RF, kept for consistency)
# ------------------------------

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ------------------------------
# 6. UNDERSAMPLING (TRAIN ONLY)
# ------------------------------

rus = RandomUnderSampler(random_state=42)
X_train_bal, y_train_bal = rus.fit_resample(X_train_scaled, y_train)

# ------------------------------
# 7. MLFLOW SETUP
# ------------------------------

mlflow.set_tracking_uri("https://dagshub.com/Pranay5519/fraud_detection.mlflow")
mlflow.set_experiment("Fraud_RandomForest_Optuna")

mlflow.sklearn.autolog()

# ------------------------------
# 8. OPTUNA OBJECTIVE FUNCTION
# ------------------------------

def objective(trial):

    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 400),
        "max_depth": trial.suggest_int("max_depth", 5, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
        "class_weight": trial.suggest_categorical(
            "class_weight",
            [{0: 1, 1: 1}, {0: 2, 1: 1}, {0: 5, 1: 1}]
        ),
        "random_state": 42,
        "n_jobs": -1
    }

    with mlflow.start_run(nested=True):

        model = RandomForestClassifier(**params)
        model.fit(X_train_bal, y_train_bal)

        y_test_pred = model.predict(X_test_scaled)

        # ---------- Classification Report ----------
        report = classification_report(
            y_test, y_test_pred, output_dict=True, zero_division=0
        )

        for label, metrics in report.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Optuna optimization target → fraud recall
        return report["1"]["precision"]

# ------------------------------
# 9. RUN OPTUNA STUDY
# ------------------------------

with mlflow.start_run(run_name="RandomForest_Run"):

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)

    mlflow.log_params({f"best_{k}": v for k, v in study.best_params.items()})

    best_params = study.best_params

    # --------------------------
    # 10. TRAIN FINAL MODEL
    # --------------------------

    final_model = RandomForestClassifier(
        **best_params,
        random_state=42,
        n_jobs=-1
    )

    final_model.fit(X_train_bal, y_train_bal)

    y_test_pred = final_model.predict(X_test_scaled)

    final_report = classification_report(
        y_test, y_test_pred, output_dict=True, zero_division=0
    )

    for label, metrics in final_report.items():
        if isinstance(metrics, dict):
            for metric, value in metrics.items():
                mlflow.log_metric(f"final_{label}_{metric}", value)

    print("Best Params:", best_params)


d:\accredian\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026/01/17 09:25:35 INFO mlflow.tracking.fluent: Experiment with name 'Fraud_RandomForest_Optuna' does not exist. Creating a new experiment.
[I 2026-01-17 09:25:37,363] A new study created in memory with name: no-name-5659941a-2993-4da3-a865-9bcc4385b8da
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 1, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 2, 1: 1} which is of type dic

🏃 View run valuable-ray-410 at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/4/runs/313d7f19facd4e7a97fbff4c2c891f1c
🧪 View experiment at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/4


[I 2026-01-17 09:26:06,866] Trial 0 finished with value: 0.03346670770493485 and parameters: {'n_estimators': 104, 'max_depth': 12, 'min_samples_split': 9, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'class_weight': {0: 2, 1: 1}}. Best is trial 0 with value: 0.03346670770493485.
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 1, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 2, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent s

🏃 View run merciful-moose-52 at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/4/runs/471e60c77fe04878b4b5ee36ae45367b
🧪 View experiment at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/4


[I 2026-01-17 09:26:44,807] Trial 1 finished with value: 0.09868967721316714 and parameters: {'n_estimators': 280, 'max_depth': 15, 'min_samples_split': 9, 'min_samples_leaf': 19, 'max_features': 'sqrt', 'class_weight': {0: 5, 1: 1}}. Best is trial 1 with value: 0.09868967721316714.
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 1, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 2, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent 

...

🏃 View run carefree-toad-427 at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/4/runs/d1e3ed54320a4dd6805c1cf91f7b8e2a
🧪 View experiment at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/4


[I 2026-01-17 09:31:10,811] Trial 8 finished with value: 0.03313840155945419 and parameters: {'n_estimators': 239, 'max_depth': 29, 'min_samples_split': 4, 'min_samples_leaf': 12, 'max_features': 'sqrt', 'class_weight': {0: 2, 1: 1}}. Best is trial 5 with value: 0.10067991631799163.
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 1, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 2, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent 

🏃 View run brawny-roo-990 at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/4/runs/d2fa2fe8d16348c7b0648ce8546f7992
🧪 View experiment at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/4


[I 2026-01-17 09:31:48,834] Trial 9 finished with value: 0.033398901729366444 and parameters: {'n_estimators': 299, 'max_depth': 27, 'min_samples_split': 19, 'min_samples_leaf': 12, 'max_features': 'log2', 'class_weight': {0: 2, 1: 1}}. Best is trial 5 with value: 0.10067991631799163.
2026/01/17 09:31:50 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Best Params: {'n_estimators': 288, 'max_depth': 29, 'min_samples_split': 5, 'min_samples_leaf': 16, 'max_features': 'log2', 'class_weight': {0: 5, 1: 1}}
🏃 View run RandomForest_Run at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/4/runs/a0632fefa7fb430a8b7c97b952392c24
🧪 View experiment at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/4


# LightGBM SMOTE Oversampling

In [5]:
import pandas as pd
import mlflow
import mlflow.sklearn
import optuna

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report , accuracy_score
from imblearn.over_sampling import SMOTE
import lightgbm as lgb

# ------------------------------
# 1. LOAD DATA
# ------------------------------

df = pd.read_csv(r"D:\accredian\data\cleaned_fraud.csv")
df.drop(columns=["Unnamed: 0"], inplace=True)
# ------------------------------
# 2. ENCODE CATEGORICAL COLUMN
# ------------------------------

df = pd.get_dummies(df, columns=['type'], drop_first=True)

# ------------------------------
# 3. DEFINE X AND y
# ------------------------------

X = df.drop(columns=['isFraud'])
y = df['isFraud']

# ------------------------------
# 4. TRAIN–TEST SPLIT
# ------------------------------

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# ------------------------------
# 5. SCALE FEATURES
# (not mandatory for LGBM, kept for consistency)
# ------------------------------

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ------------------------------
# 6. SMOTE (TRAIN ONLY)
# ------------------------------

smote = SMOTE(random_state=42, k_neighbors=5)
X_train_bal, y_train_bal = smote.fit_resample(X_train_scaled, y_train)

# ------------------------------
# 7. MLFLOW SETUP
# ------------------------------

mlflow.set_tracking_uri("https://dagshub.com/Pranay5519/fraud_detection.mlflow")
mlflow.set_experiment("Fraud_LightGBM_Optuna Exp2")

mlflow.lightgbm.autolog(disable=False)

# ------------------------------
# 8. OPTUNA OBJECTIVE FUNCTION
# ------------------------------

def objective(trial):

    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.5, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 20, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 70),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 100),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "class_weight": trial.suggest_categorical(
            "class_weight",
            [{0: 1, 1: 1}, {0: 2, 1: 1}, {0: 5, 1: 1}]
        ),
        "random_state": 42,
        "n_jobs": -1
    }

    with mlflow.start_run(nested=True):

        model = lgb.LGBMClassifier(**params)
        model.fit(X_train_bal, y_train_bal)

        y_test_pred = model.predict(X_test_scaled)

        # ---------- Classification Report ----------
        report = classification_report(
            y_test,
            y_test_pred,
            output_dict=True,
            zero_division=0
        )
        # log accuracy score
        mlflow.log_metric("accuracy", accuracy_score(y_test, y_test_pred))
        # ---------- Log metrics ----------
        for label, metrics in report.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Optuna optimization target → FRAUD PRECISION
        return report["1"]["precision"]

# ------------------------------
# 9. RUN OPTUNA STUDY
# ------------------------------

with mlflow.start_run(run_name="LightGBM_Run"):

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=20)

    mlflow.log_params(
        {f"best_{k}": v for k, v in study.best_params.items()}
    )

    best_params = study.best_params

    # --------------------------
    # 10. TRAIN FINAL MODEL
    # --------------------------

    final_model = lgb.LGBMClassifier(
        **best_params,
        random_state=42,
        n_jobs=-1
    )

    final_model.fit(X_train_bal, y_train_bal)

    y_test_pred = final_model.predict(X_test_scaled)

    final_report = classification_report(
        y_test,
        y_test_pred,
        output_dict=True,
        zero_division=0
    )
    # log accuracy score
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_test_pred))
    for label, metrics in final_report.items():
        if isinstance(metrics, dict):
            for metric, value in metrics.items():
                mlflow.log_metric(f"{label}_{metric}", value)
                #mlflow.log_metric(f"{label}_{metric}", value)
    print("Best Params:", best_params)


2026/01/17 11:01:35 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'd512a8c55c7e4507b78dd7f7fd47254d', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2026/01/17 11:01:35 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not have a `predict` or `transform` function, which is required in order to infer the signature
2026/01/17 11:01:37 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2026/01/17 11:01:45 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.


🏃 View run capable-mink-743 at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/6/runs/d512a8c55c7e4507b78dd7f7fd47254d
🧪 View experiment at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/6


[I 2026-01-17 11:01:48,651] A new study created in memory with name: no-name-1cf677bf-f698-4636-970f-7879848f0875
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 1, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 2, 1: 1} which is of type dict.
  warnings.warn(message)
d:\accredian\venv\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 5, 1: 1} which is of type dict.
  warnings.warn(message)


[LightGBM] [Info] Number of positive: 5083526, number of negative: 5083526
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.053244 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1030
[LightGBM] [Info] Number of data points in the train set: 10167052, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


d:\accredian\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2026/01/17 11:02:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
d:\accredian\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

🏃 View run rogue-wasp-598 at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/6/runs/8cddb9acf05e410dba18defa2c2fda25
🧪 View experiment at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/6


[I 2026-01-17 11:27:02,627] Trial 19 finished with value: 0.03199796962242786 and parameters: {'n_estimators': 412, 'learning_rate': 0.017296640985136242, 'num_leaves': 83, 'max_depth': 3, 'min_child_samples': 61, 'subsample': 0.7438999929173415, 'colsample_bytree': 0.6499109778125517, 'class_weight': {0: 1, 1: 1}}. Best is trial 9 with value: 0.2709433962264151.


[LightGBM] [Info] Number of positive: 5083526, number of negative: 5083526
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.055969 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1030
[LightGBM] [Info] Number of data points in the train set: 10167052, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166667 -> initscore=-1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

d:\accredian\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2026/01/17 11:27:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
d:\accredian\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Best Params: {'n_estimators': 354, 'learning_rate': 0.110593594614525, 'num_leaves': 212, 'max_depth': 11, 'min_child_samples': 84, 'subsample': 0.6077242239184953, 'colsample_bytree': 0.7543627379257855, 'class_weight': {0: 5, 1: 1}}
🏃 View run LightGBM_Run at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/6/runs/3138f215fd7149a7a55685a1ca5da6a3
🧪 View experiment at: https://dagshub.com/Pranay5519/fraud_detection.mlflow/#/experiments/6
